In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier

In [53]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
df = df.drop(['PassengerId','Name','Ticket','Fare', 'Cabin'], axis = 1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [55]:
df.dropna(inplace=True)

In [56]:
df['Embarked'].isnull().sum()

0

In [57]:
df.replace({'Embarked':{'C':0,'S':1,'Q':2,}},inplace=True)
df.replace({'Sex':{'male':1,'female':0}},inplace=True)
df['total_fam'] = df['SibSp'] + df['Parch']
df.head()
df = df.drop(['SibSp','Parch'], axis =1 )

In [58]:
X = np.array(df.drop(["Survived"], axis=1))
y = np.array(df["Survived"])

In [59]:
test_df = pd.read_csv('test.csv')
test_df.replace({'Embarked':{'C':0,'S':1,'Q':2,}},inplace=True)
test_df.replace({'Sex':{'male':1,'female':0}},inplace=True)
test_df['total_fam'] = test_df['SibSp'] + test_df['Parch']
test_df.head()
test_df = test_df.drop(['SibSp','Parch'], axis =1 )
test_df = test_df.drop(['Name','Ticket','Fare', 'Cabin'], axis = 1)

test_df.head()

,PassengerId,Pclass,Sex,Age,Embarked,total_fam
0,892,3,1,34.5,2,0
1,893,3,0,47.0,1,1
2,894,2,1,62.0,2,0
3,895,3,1,27.0,1,0
4,896,3,0,22.0,1,2


In [60]:
X_test = np.array(df.drop(["Survived"], axis=1))


In [61]:
cb = CatBoostClassifier(
    iterations = 100,
    diffusion_temperature= 0.7,
    learning_rate = 0.0085
)
cb.fit(X,y)


0:	learn: 0.6900685	total: 1.46ms	remaining: 145ms
1:	learn: 0.6874473	total: 1.85ms	remaining: 90.9ms
2:	learn: 0.6841341	total: 2.28ms	remaining: 73.8ms
3:	learn: 0.6829449	total: 2.65ms	remaining: 63.6ms
4:	learn: 0.6819104	total: 2.97ms	remaining: 56.4ms
5:	learn: 0.6794570	total: 3.17ms	remaining: 49.7ms
6:	learn: 0.6788503	total: 3.45ms	remaining: 45.8ms
7:	learn: 0.6761520	total: 3.78ms	remaining: 43.5ms
8:	learn: 0.6734865	total: 4.19ms	remaining: 42.3ms
9:	learn: 0.6710825	total: 4.58ms	remaining: 41.2ms
10:	learn: 0.6701798	total: 4.75ms	remaining: 38.4ms
11:	learn: 0.6675647	total: 5.07ms	remaining: 37.2ms
12:	learn: 0.6650796	total: 5.31ms	remaining: 35.5ms
13:	learn: 0.6643599	total: 5.54ms	remaining: 34ms
14:	learn: 0.6629544	total: 5.93ms	remaining: 33.6ms
15:	learn: 0.6601720	total: 6.1ms	remaining: 32ms
16:	learn: 0.6577493	total: 6.49ms	remaining: 31.7ms
17:	learn: 0.6565616	total: 6.75ms	remaining: 30.8ms
18:	learn: 0.6542476	total: 7.07ms	remaining: 30.1ms
19:	learn

In [62]:
predictions_cb = cb.predict(np.array(test_df.drop(['PassengerId'], axis=1)))

In [63]:
ada = AdaBoostClassifier()
ada.fit(X,y)

AdaBoostClassifier()

In [64]:
predictions_ada = ada.predict(np.array(test_df.drop(['PassengerId'], axis=1)))

ValueError: Input X contains NaN.
AdaBoostClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [65]:
test_df['Survived'] = predictions_cb
test_df.head()

,PassengerId,Pclass,Sex,Age,Embarked,total_fam,Survived
0,892,3,1,34.5,2,0,0
1,893,3,0,47.0,1,1,1
2,894,2,1,62.0,2,0,0
3,895,3,1,27.0,1,0,0
4,896,3,0,22.0,1,2,1


In [ ]:
test_df.filter(['PassengerId', 'Survived']).to_csv('1_titanic_cb.csv', index=False)